In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

In [7]:
# params
params = {
    "n_estimators": 1000,
    "random_state": 0,
    "max_depth": 10,
    "min_samples_split": 4,
    "n_jobs": 32,
    "oob_score": True
}
# read all data from train
train_data = []
train_label = []
for i in range(1, 6):
    tmp = pd.read_csv('./data/train%d.csv' % i)
    label = pd.read_csv('./data/label%d.csv' % i)
    train_data.append(tmp)
    train_label.append(label)
x = train_data[0]
y = train_label[0]
for i in range(4):
    x = np.vstack((x, train_data[i]))
    y = np.vstack((y, train_label[i]))
dataset = np.hstack((x, y))
kf = KFold(n_splits=5)
k_fold_data = []
for tra_idx, val_idx in kf.split(dataset):
    tra = dataset[tra_idx]
    val = dataset[val_idx]
    k_fold_data.append((tra, val))
    print(len(tra) + len(val))

9999998
9999998
9999998
9999998
9999998


In [ ]:
predict = []
for i in range(len(k_fold_data)):
    rf = RandomForestRegressor(**params)
    (train, test) = k_fold_data[i]
    rf.fit(train[:, :train.shape[1] - 1], train[:, -1])
#     score = rf.score(test[:, :test.shape[1] - 1], test[:, -1])
    all_predict = rf.predict(test[:, :test.shape[1] - 1])
    score = r2_score(test[:, -1], all_predict)
    predict.append((np.array(all_predict), score))
    print(score)

In [ ]:
ave = sum(predict[:][1]) / len(predict)
ave

In [ ]:
# predict
rf = RandomForestRegressor(**params)
rf.fit(x, y)
test = [pd.read_csv(f'./data/test{i + 1}.csv') for i in range(5)]
test_x = np.vstack([np.array(test[i]) for i in range(5)])
prediction = rf.predict(test_x)
output = pd.DataFrame(data = {'id': list(range(1, len(prediction)+1)), 'Predicted': prediction})
output.to_csv('result.csv', index=False)